In [1]:
import pandas as pd
import numpy as np
import math
import json
from statsmodels.imputation import mice
from statsmodels.imputation.mice import MICEData



In [26]:
# read in the json files
portfolio = pd.read_json('../data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('../data/profile.json', orient='records', lines=True)
transcript = pd.read_json('../data/transcript.json', orient='records', lines=True)

In [3]:
profile["became_member_on"] = pd.to_datetime(profile["became_member_on"],format="%Y%m%d")
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN


In [4]:
profile["became_member_on"].max()

Timestamp('2018-07-26 00:00:00')

In [5]:
profile.isna().sum()/profile.shape[0]*100

gender              12.794118
age                  0.000000
id                   0.000000
became_member_on     0.000000
income              12.794118
dtype: float64

In [6]:
profile["member_since"] = abs(profile["became_member_on"] - np.array(pd.Timestamp.now().date(),dtype='<M8[ns]'))
profile["member_since"] = profile["member_since"].astype(int)
profile.head()

,gender,age,id,became_member_on,income,member_since
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,142128000000000000
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,128908800000000000
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN,97632000000000000
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,134697600000000000
4,None,118,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN,127180800000000000


In [7]:
profile.head()

,gender,age,id,became_member_on,income,member_since
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,142128000000000000
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,128908800000000000
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN,97632000000000000
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,134697600000000000
4,None,118,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN,127180800000000000


In [8]:
profile.dtypes

gender                      object
age                          int64
id                          object
became_member_on    datetime64[ns]
income                     float64
member_since                 int64
dtype: object

In [9]:
import statsmodels as sm

In [10]:
#imp = mice.MICEData(profile[["income","member_since","age"]])
#fml = 'income ~  member_since + age'
#mice = mice.MICE(fml, sm.regression.linear_model.OLS, imp)
#results = mice.fit(10, 10)
#print(results.summary())

In [11]:
imp = mice.MICEData(profile[["income","member_since","age"]])
imp.set_imputer('income', formula='member_since + age')

for j in range(20):
    imp.update_all()

profile["income"] = imp.data["income"]

In [12]:
profile.head()

,gender,age,id,became_member_on,income,member_since
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,104000.0,142128000000000000
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,128908800000000000
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,81000.0,97632000000000000
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,134697600000000000
4,None,118,a03223e636434f42ac4c3df47e8bac43,2017-08-04,37000.0,127180800000000000


In [13]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [21]:
def filter_offer(r):
    if "offer id" in r.keys():
        return r["offer id"]
    else:
        return r

transcript["value"].apply(filter_offer)

0         9b98b8c7a33c4b65b9aebfe6a799e6d9
1         0b1e1539f2cc45b7b9fa7c272da2e1d7
2         2906b810c7d4411798c6938adc9daaa5
3         fafdcd668e3743c1bb461111dcafc2a4
4         4d5c57ea9a6940dd891ad53e9dbe8da0
                        ...               
306529      {'amount': 1.5899999999999999}
306530                    {'amount': 9.53}
306531                    {'amount': 3.61}
306532      {'amount': 3.5300000000000002}
306533                    {'amount': 4.05}
Name: value, Length: 306534, dtype: object

In [16]:
print(profile.shape,transcript.shape)
pd.merge(profile,transcript,left_on="id",right_on="person").shape

(17000, 6) (306534, 4)


(306534, 10)

In [34]:
portfolio.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [27]:
import numpy as np
def sort_channels(r):
    r.sort()
    if len(r) <4:
        r.append(np.nan)
    return r

portfolio["channels"] = portfolio["channels"].apply(sort_channels)

In [28]:
pd.DataFrame(portfolio.channels.tolist(), index= portfolio.index)

,0,1,2,3
0,email,mobile,social,NaN
1,email,mobile,social,web
2,email,mobile,web,NaN
3,email,mobile,web,NaN
4,email,web,NaN,None
5,email,mobile,social,web
6,email,mobile,social,web
7,email,mobile,social,NaN
8,email,mobile,social,web
9,email,mobile,web,NaN


https://stefvanbuuren.name/fimd/sec-pmm.html